In [14]:
from databases import BDS,Crm,BDSMem,PosicaoDm1, Boletador
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from funcoes_datas import FuncoesDatas
import pandas as pd
from emailer import Email, EmailLer
from pretty_html_table import build_table

In [15]:
##Puxa as classes do databases (consultas nas bases sql da empresa)
crm = Crm()
crm_fundos = Crm(load_cadfundos=True)
bds = BDS()
fdt = FuncoesDatas()
bol = Boletador()
posicao = PosicaoDm1

#retorna posição de todas as carteiras + fundos
posicao = PosicaoDm1()

#Retorna a data de hoje
hoje = bds.banco.hoje()

#Retorna as informações dos fundos
fundos = crm_fundos.fundo_cadastro()

#Retorna todos os produtos da casa
produtos= crm.product()

#Retorna a data 1 do mes atual
hoje = bds.banco.hoje()
dia = hoje + relativedelta(months=-1)
mes,ano = hoje.month,hoje.year

#Traz o primeiro dia util do mes anterior
data_inicial_bol = datetime(ano,mes,1)

In [16]:
def moeda(my_value):
    a = '{:,.2f}'.format(float(my_value))
    b = a.replace(',','v')
    c = b.replace('.',',')
    return c.replace('v','.')

In [20]:
###Controler de Insider

##Puxa as classes do databases (consultas nas bases sql da empresa)
crm = Crm()
bds = BDS()
fdt = FuncoesDatas()

## Parametros de data para puxar as boletas do sistema
hoje = bds.banco.hoje()
dia = hoje + relativedelta(months=-1)
mes,ano = hoje.month,hoje.year

# #Traz o primeiro dia util do mes anterior
# data_inicial_bol = datetime(ano,mes,1)
# data_inicial_bol= fdt.verificar_du(data_inicial_bol)


# #Traz o ultimo dia do mes
# dia_fim_bol = hoje + timedelta(-1)

#Usar só no mensal
data_inicial_bol = datetime(2024,6,24)
dia_fim_bol = datetime(2024,7,15)
mes,ano = data_inicial_bol.month,hoje.year

## Retorna todos os movimentos do mes (proc rv do crm) - PUs médios

movs = crm.movs_rv(data_inicial_bol,dia_fim_bol)
movs = movs[movs['new_statusboleta']==5]
movs = movs[movs['new_databoleta']>data_inicial_bol]

#Usar só no mensal
data_inicial_bol = datetime(2024,7,15)
dia_fim_bol = datetime(2024,8,5)
mes,ano = data_inicial_bol.month,hoje.year

## Retorna todos os movimentos do mes (proc rv do crm) - PUs médios

movs2 = crm.movs_rv(data_inicial_bol,dia_fim_bol)
movs2 = movs2[movs2['new_statusboleta']==5]
movs2 = movs2[movs2['new_databoleta']>data_inicial_bol]

movs = pd.concat([movs,movs2 ])

# Retorna apenas as colunas necessarias das movimentações
movs = movs[['new_databoleta','NomeTitular','new_tipomovimentorv','NomeProduto','new_idsistemaorigem','DataLiquidacao','Quantidade', 'PU','ValorOperacao']]

## Trazer o PU médio do dia por cliente e produto
movs['mult'] = movs['PU'] * movs['Quantidade']
movs = movs.groupby(['new_databoleta','NomeTitular','new_idsistemaorigem','new_tipomovimentorv'])[['mult','Quantidade']].sum()
movs['media preco'] = movs['mult'] / movs['Quantidade']
movs = movs.reset_index().drop(columns={'mult','Quantidade'}).replace(1,'Compra').replace(2,'Venda')
movs['new_databoleta'] = movs['new_databoleta'].apply(lambda x: datetime.date(x))
#Salva as movimentações em um excel
movs.to_excel(f'O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Controles 1LoD/Insider - Variação RV/Archive/Movimentações_Controle_Insider {hoje.strftime("%b")} {hoje.year}.xlsx')

## Retorna o preço médio 5 dias antes e 5 dias depois

data_inicial_bol = datetime(2024,7,1)
# retorna a query de preços
consulta_precos = bds.busca_preço_rv(hoje).sort_values(by=['Ação','Data']).fillna(0).reset_index(drop=True).dropna()
data_precos = data_inicial_bol + timedelta(-10)
consulta_precos = consulta_precos[consulta_precos['Data']>=data_precos]
consulta_precos['Data'] = consulta_precos['Data'].apply(lambda x: datetime.date(x))

#Salva as movimentações em um excel
consulta_precos.to_excel(f'O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Controles 1LoD/Insider - Variação RV/Archive/Preços_Insider {hoje.strftime("%b")} {hoje.year}.xlsx')


# Traz os valores no formato de linha para fazer a média por coluna (data)
pivot_acao = consulta_precos.groupby(['Data','Ação'])['Preço de Fechamento'].mean().unstack('Ação')
pivot_acao=pivot_acao.dropna(subset=['PETR3'])

#retorna a média móvel 5 dias à frente
#pivot_pmedio_posterior = pivot_acao.rolling(6,min_periods=1).mean().shift(-4)
pivot_pmedio_posterior=pivot_acao.shift(-4)
# #ind = pivot_pmedio_posterior.index
# #pivot_pmedio_posterior=pivot_pmedio_posterior.dropna(subset=['PETR3'])
# #n = pivot_pmedio_posterior.index
pivot_pmedio_posterior = pivot_pmedio_posterior.rolling(5,min_periods=4).mean()
#ind = ind[:len(n)-len(ind)]
#pivot_pmedio_posterior.index = ind

#Retorna num padrão tabela vertical
pivot_pmedio_posterior =pivot_pmedio_posterior.dropna(how= 'all', axis= 0).stack().reset_index().rename(columns={0:'Preço_médio_5+'})

#retorna a média móvel 5 dias à frente
pivot_pmedio_anterior = pivot_acao.rolling(5,min_periods=4).mean()
#Retorna num padrão tabela vertical
pivot_pmedio_anterior =pivot_pmedio_anterior.stack().reset_index().rename(columns={0:'Preço_médio_5-'})

#Unifica com a planilha de trades
preco_medio= pd.merge(left=pivot_pmedio_anterior,right=pivot_pmedio_posterior,on=['Data','Ação'],how='left')

#Unifica com a planilha de trades
insider_control = pd.merge(left=movs,right=preco_medio,left_on=['new_databoleta','new_idsistemaorigem'],right_on=['Data','Ação'],how='left')
insider_control = insider_control.drop(columns=['Data','new_idsistemaorigem']).dropna(subset=['Ação'])

#Incluir as verificações
insider_control['dif_media_-5'] = (insider_control['media preco']/insider_control['Preço_médio_5-'])-1
insider_control['dif_media_+5'] = (insider_control['Preço_médio_5+']/insider_control['media preco'])-1
insider_control['dif_media_-5']= round(insider_control['dif_media_-5']*100,2)
insider_control['dif_media_+5']  = round(insider_control['dif_media_+5']*100,2)
insider_control['verificar_dif_preço'] = insider_control.apply(lambda row : 'ok' if ((abs(row['dif_media_-5']) < 10) & (abs(row['dif_media_+5'])<10)) else 'desenquadrado',axis=1)
insider_control['verificar_insider'] = insider_control.apply(lambda row : 'desenquadrado' if ((row['dif_media_+5']>10) & (row['new_tipomovimentorv']=='Compra')) | ((row['dif_media_+5'] < -10) & (row['new_tipomovimentorv']=='Venda')) else 'ok',axis=1)


#Salvando o controle de insider do mês
insider_control.to_excel(f'O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Controles 1LoD/Insider - Variação RV/Archive/Mensal Junho.xlsx')

#Envio do e-mail 

verificar = insider_control[['new_databoleta', 'new_tipomovimentorv', 'Ação', 'media preco',
       'Preço_médio_5-', 'Preço_médio_5+', 'dif_media_-5', 'dif_media_+5','verificar_dif_preço',
       'verificar_insider']].rename(columns={'new_databoleta':'Data_Boleta','new_tipomovimentorv':'Tipo_Movimento'})

verificar = verificar[(verificar['verificar_insider']!='ok')&(verificar['verificar_dif_preço']!='ok')]
verificar = verificar[(verificar['Data_Boleta'] <= datetime.date(fdt.workday(fdt.hoje(),-6)))]

tbl1 = build_table(verificar,'blue_dark',text_align = 'center',width="300px")
data_email = "{:%d/%m/%Y}".format(hoje)
text_fundo =   f'''Prezados .</h3><br>
                 Seguem os apontamento de diferenças de preços com a média dps 5 dias anteriores e posteriores.</h3><br>
                {tbl1}.</h3><br>                                          
                '''        
subject= f'Monitoramento de operações de RV {data_email} [FLoD]'
to = ['portfolio@juliusbaer.com','marcos.sposito@juliusbaer.com','igor.davanzo@juliusbaer.com','andre.lett@juliusbaer.com','mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com']            
if len(verificar)==0:
    text_fundo = 'Não há apontamentos'
email = Email(to = to , subject = subject, text= text_fundo ,send = False)      
